In [10]:
import pandas as pd
import os

year_path = '2020'
directory = f'grouped/{year_path}'
if not os.path.exists(directory):
    os.makedirs(directory)

def calculate_week_number(date):
    first_day_of_year = pd.Timestamp(year=date.year, month=1, day=1)
    first_day_weekday = first_day_of_year.weekday() 

    if first_day_weekday in [0, 1, 2]:  
        return date.strftime('%W')
    else:  
        week_num = int(date.strftime('%W')) - 1
        return max(week_num, 0)  
    

In [11]:
for i in range(1,13):
    if i < 10:
        i = f'0{i}'
    file_s = f'Sales/{year_path}/sales_{year_path}_{i}.csv'
    df = pd.read_csv(file_s, delimiter=';', engine='pyarrow', dtype_backend='pyarrow', dtype={'CARD': 'string[pyarrow]'})

    df['DATE'] = pd.to_datetime(df['DATE'], format='%d.%m.%Y')
    df['QUANTITY'] = df['QUANTITY'].astype(str).str.replace(',', '.').astype('float32[pyarrow]')
    df['AMOUNT'] = df['AMOUNT'].astype(str).str.replace(',', '.').astype('float32[pyarrow]')
    df['GROSSAMOUNT'] = df['GROSSAMOUNT'].astype(str).str.replace(',', '.').astype('float32[pyarrow]')
    df['BONUSES_SPENT'] = df['GROSSAMOUNT'] - df['AMOUNT']

    df = df.groupby(['DATE', 'PRODUCT', 'BONUSES_SPENT', 'GROSSAMOUNT', 'AMOUNT']).agg({'QUANTITY': 'sum'}).reset_index()
    df = df.rename(columns={'PRODUCT': 'PRODUCT_GUID', 'GROSSAMOUNT': 'COST (without bonuses)'})

    nom = pd.read_csv('Products.csv', delimiter=';', engine='pyarrow', dtype_backend='pyarrow')
    nom = nom.drop('DIRECTION', axis=1)
    nom = nom.drop('ZONE', axis=1)
    nom = nom.drop('BRAND', axis=1)
    nom = nom.rename(columns={'GUID': 'PRODUCT_GUID'})

    lst_merged_df = df.set_index('PRODUCT_GUID').join(nom.set_index('PRODUCT_GUID'))
    lst_merged_df = lst_merged_df.reset_index()
    lst_merged_df['Week day'] = lst_merged_df['DATE'].dt.strftime('%a')
    lst_merged_df['Week number'] = lst_merged_df['DATE'].apply(calculate_week_number).astype(int)


    lst_merged_df['Month number'] = lst_merged_df['DATE'].dt.month
    lst_merged_df.drop(columns=['PRODUCT_GUID'], inplace=True)

    desired_order = ['DATE', 'Week number','Month number', 'Week day', 'BONUSES_SPENT','COST (without bonuses)','AMOUNT','QUANTITY','PRODUCT','CATEGORY','SUBCATEGORY']
    lst_merged_df = lst_merged_df[desired_order]

    subcategory_grouped_df = lst_merged_df.groupby(['DATE', 'CATEGORY', 'SUBCATEGORY', 'Week number', 'Month number', 'Week day']).agg({
        'QUANTITY': 'sum',
        'BONUSES_SPENT': 'sum',
        'COST (without bonuses)': 'sum'
    }).reset_index()

    subcategory_grouped_df["BONUSES_RECIVE"] = subcategory_grouped_df["COST (without bonuses)"] * 0.025
    subcategory_grouped_df["BONUSES_RECIVE"] = subcategory_grouped_df["BONUSES_RECIVE"].apply(lambda x: '{:.2f}'.format(x))
    subcategory_grouped_df.to_csv(f'grouped/{year_path}/grouped_by_subcat_{i}_{year_path}.csv', index=False)


In [12]:
df = pd.read_csv(f'grouped/{year_path}\grouped_by_subcat_01_{year_path}.csv', delimiter=',', engine='pyarrow', dtype_backend='pyarrow',  dtype={'CARD': 'string[pyarrow]'})
for i in range(2,13):
    if i < 10:
        i = f'0{i}'
    file_s = f'grouped/{year_path}/grouped_by_subcat_{i}_{year_path}.csv'
    df = pd.concat([df, pd.read_csv(file_s, delimiter=',', engine='pyarrow', dtype_backend='pyarrow',  dtype={'CARD': 'string[pyarrow]'})], ignore_index=True)

df_sorted = df.sort_values(by=['CATEGORY', 'SUBCATEGORY', 'Week number'])
df_sorted.to_csv(f'grouped/{year_path}/grouped_by_subcat_{year_path}.csv', index=False)
